In [2]:
#This is a Stationary Dataset.
#The first two columns of the dataset are features.
#The last column of the dataset is the label.
#The 2nd and 5th row is written in a way that they both have similar features but different labels,
#to see how the model handles this case.
training_data = [
    ['Blue', 10, 'Pen'],
    ['Black', 10, 'Pen'],
    ['White', 3, 'Eraser'],
    ['White', 3, 'Eraser'],
    ['Black', 10, 'Pencil'],
]

In [3]:
#This is the header of the Dataset.
header = ["color", "length", "label"]

In [4]:
#This Function will find unique values for a column in the dataset.
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [5]:
unique_vals(training_data, 0)

{'Black', 'Blue', 'White'}

In [6]:
#This function counts the number of each type of items in the dataset.
def class_counts(rows):
    counts = {}  
    for row in rows:
        label = row[-1] #because in this dataset, label is the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
class_counts(training_data)

{'Pen': 2, 'Eraser': 2, 'Pencil': 1}

In [8]:
#This function tests if the value is numeric or not.
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [9]:
is_numeric(7)

True

In [10]:
#The class Question is used to partition the dataset. 
class Question:

    def __init__(self, column, value):
        self.column = column
        self.value = value
#The method match is used to comapre the features in each row with the features stored in the class.
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
#The method __repr__ is used to print the question being asked. 
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [11]:
Question(1, 10)

Is length >= 10?

In [12]:
q = Question(0, 'Blue')
q

Is color == Blue?

In [13]:
example = training_data[0]
q.match(example) 

True

In [14]:
#The method partition checks if any row matches to the question.
#If so, the item is added to the true rows or else to the false rows.
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [15]:
#Here, we are Partitioning the dataset on the basis of whether the item is White in colour or not.
true_rows, false_rows = partition(training_data, Question(0, 'White'))
#True_rows will contaions all the rows of items that are White.
true_rows

[['White', 3, 'Eraser'], ['White', 3, 'Eraser']]

In [16]:
#false_rows will conatin all the rows of items that are not White.
false_rows

[['Blue', 10, 'Pen'], ['Black', 10, 'Pen'], ['Black', 10, 'Pencil']]

In [17]:
#The method gini calculates GINI impurities for all the rows in the datset.
#GINI impurity is used to decide the optimal split from a root node in a decision tree. 
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [18]:
#An example with no mixing gives out the value of GINI impurity as 0
no_mixing = [['Pen'],
              ['Pen']]
gini(no_mixing)

0.0

In [19]:
#An example with 50% mixing gives 0.5
some_mixing = [['Pen'],
               ['Ruler']]
gini(some_mixing)

0.5

In [20]:
#An example with lots of mixing. 
lots_of_mixing = [['Pen'],
                  ['Ruler'],
                  ['Eraser'],
                  ['Sharpner'],
                  ['Marker']]
gini(lots_of_mixing)

0.7999999999999998

In [21]:
#The method info_gain will measure uncertainty of the starting node, minus the weighted impurity of two sub-nodes.
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [22]:
#Calculating Uncertaininty of the training datset.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [23]:
#How much information is gained when partitioned by the colour 'Blue'?
true_rows, false_rows = partition(training_data, Question(0, 'Blue'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [24]:
#How much information is gained when partioned by the colour 'White'?
true_rows, false_rows = partition(training_data, Question(0,'White'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [25]:
#Since, information gained by the colour 'White'(0.373) is more then that gained by 'Blue'(0.139) 
true_rows, false_rows = partition(training_data, Question(0,'White'))
#Here, the true_rows conatins only one type of item i.e. 'Eraser'
true_rows

[['White', 3, 'Eraser'], ['White', 3, 'Eraser']]

In [26]:
#And false_rows conatains two types of items. 
false_rows

[['Blue', 10, 'Pen'], ['Black', 10, 'Pen'], ['Black', 10, 'Pencil']]

In [27]:
#But when we partitioned the data using 'Blue' colour 
#true_rows contains only one item i.e. 'Pen'
true_rows, false_rows = partition(training_data, Question(0,'Blue'))
true_rows

[['Blue', 10, 'Pen']]

In [28]:
#But false_rows conatins alot of items. 
false_rows

[['Black', 10, 'Pen'],
 ['White', 3, 'Eraser'],
 ['White', 3, 'Eraser'],
 ['Black', 10, 'Pencil']]

In [29]:
#The method find_best_split will find the best question to ask by iterating every feature and calculating the information gain.
def find_best_split(rows):
    best_gain = 0  #keeps track of the best information gain
    best_question = None  #keeps train of the feature that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  #number of columns

    for col in range(n_features):  #for each feature

        values = set([row[col] for row in rows]) #unique values in the column


        for val in values:  #for each value

            question = Question(col, val)
            #tries splitting the dataset  
            true_rows, false_rows = partition(rows, question)
            #Skips this split if it doesn't divide thevdataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            #Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [30]:
#Finding the best question for our training dataset
best_gain, best_question = find_best_split(training_data)
best_question

Is length >= 10?

In [31]:
#The class Leaf will hold a dictionary of class i.e. number of times
#it appears in the rows from the training data that reach this leaf.
class Leaf:

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [32]:
#This class holds a reference to the question, and to the two sub-nodes.
class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [33]:
#The method build_tree builds the tree. 
def build_tree(rows):
#After partitioing the dataset on each of the unique attribute, calculating the information gain,
#and returning the question that produces the highest gain.
    gain, question = find_best_split(rows)
#Base case: no further info gain, we can ask no further questions, we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    #If we reach here, we have found a useful feature to partition.
    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)#Recursively builds the true branch.

    false_branch = build_tree(false_rows)#Recursively builds the false branch.
    #Returns a Question node, this records the best feature to ask at this point,
    #as well as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [34]:
#This method will print the tree.
def print_tree(node, spacing=""):

    if isinstance(node, Leaf):#Base case: we've reached a leaf
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))#Printing the question at this node

    print (spacing + '--> True:')#Calling this function recursively on the true branch
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')#Calling this function recursively on the false branch
    print_tree(node.false_branch, spacing + "  ")

In [35]:
my_tree = build_tree(training_data)

In [36]:
print_tree(my_tree)

Is length >= 10?
--> True:
  Is color == Black?
  --> True:
    Predict {'Pen': 1, 'Pencil': 1}
  --> False:
    Predict {'Pen': 1}
--> False:
  Predict {'Eraser': 2}


In [37]:
def classify(row, node):

    if isinstance(node, Leaf):#Base case: we've reached a leaf
        return node.predictions

    #Decides whether to follow the true-branch or the false-branch,
    #Compares the feature stored in the node, to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [38]:
#The tree preditcs the first row of the dataset with item 'Pen' and confidence 1
classify(training_data[0], my_tree)

{'Pen': 1}

In [39]:
#This method will print the prediction of the leaf.
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [40]:
print_leaf(classify(training_data[0], my_tree))

{'Pen': '100%'}

In [41]:
#For the second row of the dataset, the confidence is low, i.e; 0.5
print_leaf(classify(training_data[1], my_tree))

{'Pen': '50%', 'Pencil': '50%'}

In [42]:
#This is our testing Dataset.
testing_data = [
    ['Blue', 10, 'Pen'],
    ['Black', 10, 'Pen'],
    ['White', 3, 'Eraser'],
    ['White', 2, 'Eraser'],
    ['Black', 10, 'Pencil'],
    ['Black' , 10, 'Pen'] 
]

In [43]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Pen. Predicted: {'Pen': '100%'}
Actual: Pen. Predicted: {'Pen': '50%', 'Pencil': '50%'}
Actual: Eraser. Predicted: {'Eraser': '100%'}
Actual: Eraser. Predicted: {'Eraser': '100%'}
Actual: Pencil. Predicted: {'Pen': '50%', 'Pencil': '50%'}
Actual: Pen. Predicted: {'Pen': '50%', 'Pencil': '50%'}
